In [1]:
import pandas as pd

# Load the CSV file
file_path = "matching_data/ror_id.csv"
df = pd.read_csv(file_path)

# Replace "Russia" with "Russian Federation" in the "Country Name" column
df["Country Name"] = df["Country Name"].replace("Russia", "Russian Federation")

# Save the modified CSV
df.to_csv(file_path, index=False)

print("Preprocessing complete: 'Russia' replaced with 'Russian Federation'.")


Preprocessing complete: 'Russia' replaced with 'Russian Federation'.


Method 1, exact matching only

In [19]:
import csv

# Step 1: Read ror_id.csv and create the hashmap
ror_map = {}

with open('matching_data/ror_id.csv', mode='r', encoding='utf-8') as ror_file:
    ror_reader = csv.DictReader(ror_file)
    for row in ror_reader:
        country = row['Country Name']
        institution = row['Institution Name']
        ror_id = row['ROR ID']
        
        if country not in ror_map:
            ror_map[country] = {}
        ror_map[country][institution] = ror_id

# Step 2: Read scopus_id.csv and match ROR IDs
matched_data = []

with open('matching_data/scopus_id.csv', mode='r', encoding='utf-8') as scopus_file:
    scopus_reader = csv.DictReader(scopus_file)
    for row in scopus_reader:
        country = row['Country Name']
        primary_org = row['Primary Org Name']
        scopus_id = row['Scopus ID']
        
        ror_id = None
        if country in ror_map and primary_org in ror_map[country]:
            ror_id = ror_map[country][primary_org]
        
        matched_data.append({
            'Scopus ID': scopus_id,
            'Primary Org Name': primary_org,
            'Country Name': country,
            'ROR ID': ror_id
        })

# Step 3: Save the matched data to a new CSV file
with open('matching_data/matched_results.csv', mode='w', encoding='utf-8', newline='') as output_file:
    fieldnames = ['Scopus ID', 'Primary Org Name', 'Country Name', 'ROR ID']
    writer = csv.DictWriter(output_file, fieldnames=fieldnames)
    
    writer.writeheader()
    for row in matched_data:
        writer.writerow(row)

print("Matching completed. Results saved to 'matching_data/matched_results.csv'.")

Matching completed. Results saved to 'matching_data/matched_results.csv'.


Method 2, exact matching combined with elastic searching

In [ ]:
import csv
from fuzzywuzzy import process
from tqdm import tqdm  # Import tqdm for progress bars

# Step 1: Read ror_id.csv and create the hashmap
ror_map = {}

with open('matching_data/ror_id.csv', mode='r', encoding='utf-8') as ror_file:
    ror_reader = csv.DictReader(ror_file)
    for row in tqdm(ror_reader, desc="Reading ROR data"):  # Add progress bar
        country = row['Country Name']
        institution = row['Institution Name']
        ror_id = row['ROR ID']
        
        if country not in ror_map:
            ror_map[country] = {}
        ror_map[country][institution] = ror_id

# Step 2: Read scopus_id.csv and match ROR IDs
matched_data = []

with open('matching_data/scopus_id.csv', mode='r', encoding='utf-8') as scopus_file:
    scopus_reader = csv.DictReader(scopus_file)
    for row in tqdm(scopus_reader, desc="Matching Scopus data"):  # Add progress bar
        country = row['Country Name']
        primary_org = row['Primary Org Name']
        scopus_id = row['Scopus ID']
        
        ror_id = None
        if country in ror_map:
            # Try exact match first
            if primary_org in ror_map[country]:
                ror_id = ror_map[country][primary_org]
            else:
                # If exact match fails, try fuzzy matching
                choices = list(ror_map[country].keys())
                best_match, score = process.extractOne(primary_org, choices)
                print(best_match)
                if score >= 80:  # You can adjust the threshold as needed
                    ror_id = ror_map[country][best_match]
        
        matched_data.append({
            'Scopus ID': scopus_id,
            'Primary Org Name': primary_org,
            'Country Name': country,
            'ROR ID': ror_id
        })

# Step 3: Save the matched data to a new CSV file
with open('matching_data/matched_results.csv', mode='w', encoding='utf-8', newline='') as output_file:
    fieldnames = ['Scopus ID', 'Primary Org Name', 'Country Name', 'ROR ID']
    writer = csv.DictWriter(output_file, fieldnames=fieldnames)
    
    writer.writeheader()
    for row in tqdm(matched_data, desc="Writing results"):  # Add progress bar
        writer.writerow(row)

print("Matching completed. Results saved to 'matching_data/matched_results_elastic.csv'.")

In [1]:
import pandas as pd

# Load the CSV file
file_path = "matching_data/matched_results_fuzzy.csv"
df = pd.read_csv(file_path)

# Create a mask for rows where 'ROR ID' is empty (NaN or only whitespace)
empty_mask = df["ROR ID"].isna() | (df["ROR ID"].astype(str).str.strip() == "")
empty_count = empty_mask.sum()

# Calculate the total number of rows
total_rows = len(df)

# Calculate the percentage of rows with empty ROR ID
percentage_empty = (empty_count / total_rows * 100) if total_rows > 0 else 0

print("Number of rows with empty ROR ID:", empty_count)
print("Percentage of rows with empty ROR ID: {:.2f}%".format(percentage_empty))


Number of rows with empty ROR ID: 200
Percentage of rows with empty ROR ID: 1.50%
